# UK Biobank RAP - Basic data extraction


This notebook is delivered "As-Is". Notwithstanding anything to the contrary, DNAnexus will have no warranty, support or other obligations with respect to Materials provided hereunder.

[MIT License](https://github.com/dnanexus/OpenBio/blob/master/LICENSE.md) applies to this notebook.

### Prologue

This prologue has been created for consistency across notebooks, and to improve notebook portability across projects.

In [ ]:
# Import packages
import pyspark
import dxpy
import dxdata

In [ ]:
# Spark initialization (Done only once; do not rerun this cell unless you select Kernel -> Restart kernel).
sc = pyspark.SparkContext()
spark = pyspark.sql.SparkSession(sc)

In [ ]:
# Automatically discover dispensed database name and dataset id
dispensed_database_name = dxpy.find_one_data_object(classname="database", name="app*", folder="/", name_mode="glob", describe=True)["describe"]["name"]
dispensed_dataset_id = dxpy.find_one_data_object(typename="Dataset", name="app*.dataset", folder="/", name_mode="glob")["id"]

## Access dataset

In [ ]:
dataset = dxdata.load_dataset(id=dispensed_dataset_id)

### Dataset "entities" are virtual tables linked to one another.

The main entity is "participant" and corresponds to most pheno fields. Additional entities correspond to linked health care data.
Entities starting with "hesin" are for hospital records; entities starting with "gp" are for GP records, etc.

In [ ]:
dataset.entities

### Accessing the main 'participant' entity

In [ ]:
participant = dataset["participant"]

#### Selecting participant fields by field index, instance index, array index

For the main participant data, the Platform uses field names with the following convention:

|Type of field|Syntax for field name|Example|
|:------------|---------------------|-------|
|Neither instanced nor arrayed|`p<FIELD-ID>`|`p31`|
|Instanced but not arrayed|`p<FIELD-ID>_i<INSTANCE-ID>`|`p40005_i0`|
|Arrayed but not instanced|`p<FIELD-ID>_a<ARRAY-ID>`|`p41262_a0`|
|Instanced and arrayed|`p<FIELD-ID>_i<INSTANCE-ID>_a<ARRAY-ID>`|`p93_i0_a0`|

Lastly, the participant id field itself (EID) is named `eid`

If you know exactly the field names you want to work with, put them in a string array (we will see later how to use that):

In [ ]:
field_names = ["eid", "p31", "p21022", "p40005_i0", "p93_i0_a0"]

#### Looking up fields by id

If you know the field id but you are not sure if it is instanced or arrayed, and want to grab all instances/arrays (if any), use these:

In [ ]:
# Returns all fields for a given UK Biobank data-field id

def fields_for_id(field_id):
    from distutils.version import LooseVersion
    field_id = str(field_id)
    fields = participant.find_fields(name_regex=r'^p{}(_i\d+)?(_a\d+)?$'.format(field_id))
    return sorted(fields, key=lambda f: LooseVersion(f.name))

# Returns all field names for a given UK Biobank data-field id

def field_names_for_id(field_id):
    return [f.name for f in fields_for_id(field_id)]

##### Examples:

In [ ]:
# Participant sex
field_names_for_id("31")

In [ ]:
# Age when attending assessment centre has multiple instances (visits) 
field_names_for_id("21003")

In [ ]:
# Pulse rate has multiple instances and array indices (measured twice in each visit)
field_names_for_id("102")

#### Looking up fields by title keyword

If you remember part of the field title, use these:

In [ ]:
# Returns all field objects for a given title keyword

def fields_by_title_keyword(keyword):
    from distutils.version import LooseVersion
    fields = list(participant.find_fields(lambda f: keyword.lower() in f.title.lower()))
    return sorted(fields, key=lambda f: LooseVersion(f.name))

# Returns all field names for a given title keyword

def field_names_by_title_keyword(keyword):
    return [f.name for f in fields_by_title_keyword(keyword)]

# Returns all field titles for a given title keyword

def field_titles_by_title_keyword(keyword):
    return [f.title for f in fields_by_title_keyword(keyword)]

In [ ]:
field_titles_by_title_keyword('standing height')

In [ ]:
field_names_by_title_keyword('standing height')

#### You can mix and match these methods to end up with a list of field names of interest:

In [ ]:
field_names = ["eid", "p31", "p21022"] + field_names_for_id("41262") + field_names_by_title_keyword('diagnoses - main')

### Grabbing fields into a spark dataframe

`retrieve_fields` function allows you to get subset of data into tabular format. 

You can set `coding_values` parameter as:
- "raw" (default: leave coded values as they are)
- "exclude" (if sparse-coded, treat codes as missing data by replacing with null)
- "replace" (replace coded values with their meanings)

In [ ]:
df = participant.retrieve_fields(names=field_names, engine=dxdata.connect())

In [ ]:
# See the first five entries as a Spark DataFrame:
df.show(5, truncate=False)

In [ ]:
# See the first five entries as a Pandas DataFrame:
df.limit(5).toPandas()

### Filtering spark dataframes

Spark dataframes can be filtered using the syntax: `df.filter(expression)`

The expression can be either :

* a string expression, built using SQL fields (e.g. `p31`) and SQL operators (e.g. `=`, `NOT`, `OR`, `AND`)
  * example: `"(p21022 >= 50) AND (p31 = 0)"`
  

* a Python expression, built using Python object fields (e.g. `df.p31`) and Python operators (e.g. `==`, `!`, `|`, `&`)
  * example: `(df.p21022 >= 50) & (df.p31 == 0)`

#### Example: Participants above 50 years old and female

In [ ]:
df.count()

In [ ]:
# Using SQL syntax
df.filter("(p21022 >= 50) AND (p31 = 0)").count()

In [ ]:
# Using Python syntax
df.filter((df.p21022 >= 50) & (df.p31 == 0)).count()

#### Working with codings

In [ ]:
participant["p31"].coding

In [ ]:
participant["p31"].coding.codes

In [ ]:
def field_codes_by_keyword(field_name, keyword):
    return dict([(k,v) for (k,v) in participant[field_name].coding.codes.items() if keyword.lower() in v.lower()])

In [ ]:
field_codes_by_keyword("p31", "female")

In [ ]:
field_codes_by_keyword("p41202", "obesity")

### Getting information about the fields

In [ ]:
# get link to UKB documentation page
participant["p31"].linkout

In [ ]:
# Get field units
participant["p21022"].units

### Saving results

In [ ]:
# Saving as CSV file
df.toPandas().to_csv('results.csv', index=False)

In [ ]:
# Saving as TSV file
df.toPandas().to_csv('results.tsv', sep='\t', index=False)

In [ ]:
# Saving as DTA file (Stata)
df.toPandas().astype(str).replace('None|NaN|nan', '.', regex=True).to_stata('results.dta')

#### Writing results back to the project

In [ ]:
%%bash
dx upload results.tsv --dest / 